# Creating, Planting, and Finding Signals Within Noise
Elizabeth Kapelevich

In [18]:
import time
import math
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import signals_and_noise as san

In [19]:
Time1, Position1 = np.loadtxt("mock_data_challenge_1.dat").T
Time2, Position2 = np.loadtxt("mock_data_challenge_2.dat").T

## Main Goal for Week of 7/04:
* Begin PyCBC and learn implementation
### Mini-goal:
* Adjust functions `template`, `integrator`, `cross_correlation`, and `frequency_time_search` to allow $\sigma$ (standard deviation) as an array and output the corresponding value that relates to the cross-correlation and frequency.

In [23]:
def template(a, f, sigma, t_0, t_duration, data_time_stamps, ad_hoc_grid = 10000):
    
    del_T_data = np.diff(data_time_stamps)[0]   
    t_end = t_duration + t_0
    time_stamps = np.linspace(t_0, t_end + del_T_data, ad_hoc_grid)   
    
    t_mean = np.mean(time_stamps)
    t_stamps, frequency = np.meshgrid(time_stamps, f)    

    S = a*np.sin(2*np.pi*frequency*t_stamps)*np.exp((-(t_stamps - t_mean)**2)/(2*sigma))

    all_templates = []        
    
    for i in range(len(f)):
        s = S[i]
        interpolant = interpolate.interp1d(time_stamps, s)
        Template = interpolant(data_time_stamps[(data_time_stamps >= t_0)*(data_time_stamps <= t_end)])

        template_series = np.zeros_like(data_time_stamps)   

        index_prefix = template_series[data_time_stamps < t_0]
        index_suffix = template_series[data_time_stamps > t_end]  

        closest_to_t0 = data_time_stamps[np.argmin(np.abs(t_0 - data_time_stamps))] 
        closest_to_tend = data_time_stamps[np.argmin(np.abs(t_end - data_time_stamps))]

        result = np.hstack((index_prefix, Template, index_suffix))  
        all_templates.append(result) 
        
    return all_templates

In [24]:
start = time.time()
my_temp = []

f = np.linspace(0.1, 10, 500)

for sigma in np.linspace(0.1, 2, 500):
    my_temp.append(template(2.5, f, sigma, 2, 8, Time1))

end = time.time()
print(end - start)

94.67747473716736


In [15]:
a = 1.5
f = np.linspace(0, 10, 20)
sigma = np.linspace(0, 1, 10)
t_0 = 2.01
t_start = 2
t_duration = 8
t_max = 20
data_time_series = [Time1, Position1]
data_time_stamps = Time1
del_T_0 = 0.012
del_T = 0.012

In [16]:
my_template = template(a, f, sigma, t_0, t_duration, data_time_stamps)

ValueError: operands could not be broadcast together with shapes (20,10000) (10,) 